# SafeAI in 60 Seconds

Two lines to set up. Then wrap any AI API call — Gemini, OpenAI, Claude, anything.

```python
from safeai import SafeAI
ai = SafeAI.quickstart()   # that's it
```

In [1]:
from safeai import SafeAI

ai = SafeAI.quickstart()
print("SafeAI ready.")

SafeAI ready.


## 1. Quick sanity check — does it catch secrets and PII?

In [2]:
# Safe text passes through
result = ai.scan_input("What is the capital of France?")
print(f"Clean input:  {result.decision.action}")  # allow

# API keys are blocked
result = ai.scan_input("Use this key: sk-ABCDEF1234567890ABCDEF")
print(f"Secret input: {result.decision.action} — {result.decision.reason}")  # block

# PII is redacted in outputs
result = ai.guard_output("Contact alice@example.com or call 555-123-4567")
print(f"PII output:   {result.decision.action}")
print(f"Safe text:    {result.safe_output}")  # emails/phones replaced with [REDACTED]

Clean input:  allow
Secret input: block — Secrets must never cross any boundary.
PII output:   redact
Safe text:    Contact [REDACTED] or call [REDACTED]


## 2. Real API call — Gemini with SafeAI guardrails

This wraps an actual Gemini API call. SafeAI scans the prompt before it leaves, and guards the response before you use it.

Set your `GOOGLE_API_KEY` env var or paste it below.

In [3]:
import os
from google import genai

# Set your key via GOOGLE_API_KEY env var
API_KEY = os.environ.get("GOOGLE_API_KEY", "")
if not API_KEY:
    print("No GOOGLE_API_KEY found — set it to run the live Gemini demo.")
    print("Skipping Gemini cells. SafeAI itself works fine (see above).")
    SKIP_GEMINI = True
else:
    client = genai.Client(api_key=API_KEY)
    SKIP_GEMINI = False
    print("Gemini client ready.")


def safe_generate(prompt: str, model: str = "gemini-2.0-flash") -> str:
    """Call Gemini with SafeAI guardrails — scan input, guard output."""

    # 1. Check the prompt before sending
    scan = ai.scan_input(prompt)
    if scan.decision.action == "block":
        return f"BLOCKED by SafeAI: {scan.decision.reason}"

    # 2. Call Gemini
    response = client.models.generate_content(model=model, contents=prompt)

    # 3. Guard the response before returning
    guard = ai.guard_output(response.text)
    return guard.safe_output  # PII redacted, secrets stripped


if not SKIP_GEMINI:
    answer = safe_generate("Explain what an API key is in one sentence.")
    print("Gemini says:", answer)
else:
    print("(skipped — no API key)")

No GOOGLE_API_KEY found — set it to run the live Gemini demo.
Skipping Gemini cells. SafeAI itself works fine (see above).
(skipped — no API key)


In [4]:
# Now try to sneak a secret through — SafeAI blocks it before it reaches Gemini
if not SKIP_GEMINI:
    answer = safe_generate("Summarize this config: API_KEY=sk-ABCDEF1234567890ABCDEF")
    print("Result:", answer)
else:
    # Demo the blocking even without Gemini — this is purely SafeAI
    scan = ai.scan_input("Summarize this config: API_KEY=sk-ABCDEF1234567890ABCDEF")
    print(f"Result: BLOCKED by SafeAI: {scan.decision.reason}")

Result: BLOCKED by SafeAI: Secrets must never cross any boundary.


In [5]:
# Ask Gemini something that might return PII — SafeAI redacts it in the output
if not SKIP_GEMINI:
    answer = safe_generate("Make up a fake customer profile with name, email, and phone number.")
    print("Guarded output:\n", answer)
else:
    # Demo the redaction even without Gemini
    fake_response = "Name: Alice Smith, Email: alice@corp.io, Phone: 555-867-5309"
    guard = ai.guard_output(fake_response)
    print(f"Guarded output:\n {guard.safe_output}")

Guarded output:
 Name: Alice Smith, Email: [REDACTED], Phone: [REDACTED]


## 3. Configure what SafeAI enforces

`quickstart()` accepts keyword arguments to toggle protections on/off:

| Argument | Default | Effect |
|---|---|---|
| `block_secrets` | `True` | Block API keys, tokens, credentials |
| `redact_pii` | `True` | Redact emails, phones, SSNs in outputs |
| `block_pii` | `False` | Block PII entirely instead of redacting |
| `custom_rules` | `None` | Add your own policy rules (list of dicts) |

In [6]:
# Example: block PII entirely (not just redact)
strict = SafeAI.quickstart(block_pii=True, redact_pii=False)

result = strict.guard_output("Contact alice@example.com")
print(f"block_pii=True:  {result.decision.action}")  # block (not redact)

# Example: secrets only, ignore PII
secrets_only = SafeAI.quickstart(redact_pii=False)

result = secrets_only.guard_output("Contact alice@example.com")
print(f"redact_pii=False: {result.decision.action}")  # allow (PII passes through)

result = secrets_only.scan_input("token=sk-ABCDEF1234567890ABCDEF")
print(f"secrets still blocked: {result.decision.action}")  # block

block_pii=True:  block
redact_pii=False: allow
secrets still blocked: block


In [7]:
# Example: add a custom rule — block anything tagged "financial" on input
custom = SafeAI.quickstart(custom_rules=[
    {
        "name": "block-financial-input",
        "boundary": ["input"],
        "priority": 15,
        "condition": {"data_tags": ["personal.financial"]},
        "action": "block",
        "reason": "Financial data cannot be sent to the model.",
    },
])

result = custom.scan_input("Card: 4111-1111-1111-1111")
print(f"Custom rule blocks credit card: {result.decision.action}")  # block

Custom rule blocks credit card: block


## That's it.

- `SafeAI.quickstart()` — zero config, works immediately
- `ai.scan_input(text)` — check prompts before sending to any LLM
- `ai.guard_output(text)` — redact PII/secrets from any LLM response
- Toggle `block_secrets`, `redact_pii`, `block_pii` to control what gets enforced
- Pass `custom_rules` for anything beyond the built-in protections
- Works with **any** AI provider: Gemini, OpenAI, Claude, Ollama, etc.